In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
from pathlib import Path
import anndata as ad
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "/home/ylu/project/spateo-release_new")
import spateo_dev as st
sys.path.insert(0, "/home/ylu/project")
from utils import *

os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


(EE) 
Fatal server error:
(EE) Server is already active for display 99
	If this server is no longer running, remove /tmp/.X99-lock
	and start again.
(EE) 


In [5]:
data_folder = "../../data/BARseq/BARseq_Perturbed/"
# Path(results_folder).mkdir(parents=True, exist_ok=True)
method = 'PASTE'

In [6]:
slices = [ad.read_h5ad(data_folder + "slice_{:0>2d}.h5ad".format(i)) for i in range(1, 41)]

In [7]:
transformation = np.load('./results/processed_results/PASTE_transformation.npy', allow_pickle=True).item()
align_Rotation = transformation['align_Rotation']
align_translation = transformation['align_translation']

In [8]:
slices[0]

AnnData object with n_obs × n_vars = 34029 × 114
    obs: 'cluster', 'slice_id', 'simplify_cluster'
    obsm: 'align_spatial', 'perturbed_spatial', 'spatial', 'spatial_CCF', 'spatial_raw'

In [9]:
from tqdm import tqdm
cur_R = np.eye(2)
cur_t = np.zeros(2)
for i in tqdm(range(len(slices))):
    slices[i].obsm['align_spatial'] = slices[i].obsm['perturbed_spatial'] @ cur_R.T + cur_t
    if i < len(slices)-1:
        R = align_Rotation[i]
        t = align_translation[i]
        cur_t = t @ cur_R.T + cur_t
        cur_R = cur_R @ R

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 582.08it/s]


In [10]:
z_values = []
for i in range(len(slices)):
    z_values.append(np.mean(slices[i].obsm['spatial_CCF'][:,2]))

In [11]:
import pyvista as pv
brain_mesh = pv.read('../../data/HalfAdultMouseBrain.vtk')

In [12]:
brain_mesh_align = st.tdr.scale_model(model=brain_mesh, scale_factor=40, scale_center=None, inplace=False)
brain_mesh_align = st.tdr.rotate_model(model=brain_mesh_align, angle=(0, 0, 90), rotate_center= brain_mesh.center, inplace=False)

In [13]:
z_shift_correction = st.align.z_shift_correction(
    models=slices,
    mesh=brain_mesh_align,
    spatial_key='align_spatial',
    key_added='align_spatial_z_corrected',
    z_values=z_values,
)

In [14]:
z_shift_correction.extract_contours(method='alpha', smoothing=True, window_size=3, outlier_threshold=30, alpha=0.2, n_sampling=10000)

|-----> [Extract contours of slices, method: alpha.] in progress: 100.0000%
|-----> [Extract contours of slices, method: alpha.] finished [78.7713s]


In [ ]:
init_parameters = np.array([0,0,0,0,1])
best_parameters, _ = z_shift_correction.start_discrete_optimization(
    init_parameters=init_parameters,
    max_rotation_angle=5,
    max_translation_scale=0.1,
    max_scaling=1.05,
    labelNum=8,
    nIters=10,
    multi_process=True,
)

Current iter: 0, parameters: [0 0 0 0 1] with loss: 100000000.0
evaluate 10 pairs
Current pair: 0/10


/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:184: RuntimeWarning:

invalid value encountered in divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:184: RuntimeWarning:

invalid value encountered in divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/home/ylu/anaconda3/envs/

Current pair: 1/10


/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:184: RuntimeWarning:

invalid value encountered in divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:184: RuntimeWarning:

invalid value encountered in divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/home/ylu/anaconda3/envs/

Current pair: 2/10


...

Updating parameters!
Current iter: 2, parameters: [-10.71428571   6.42857143  -3.57142857 -53.58979562   0.95268987] with loss: 0.2715090060464961
evaluate 10 pairs
Current pair: 0/10
Current pair: 1/10
Current pair: 2/10
Current pair: 3/10
Current pair: 4/10
Current pair: 5/10
Current pair: 6/10
Current pair: 7/10
Current pair: 8/10
Current pair: 9/10


/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:184: RuntimeWarning:

invalid value encountered in divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide



Updating parameters!
Current iter: 3, parameters: [ -0.71428571  10.71428571  -2.14285714 -49.89134494   0.96575015] with loss: 0.19780876968510344
evaluate 10 pairs
Current pair: 0/10
Current pair: 1/10
Current pair: 2/10
Current pair: 3/10
Current pair: 4/10
Current pair: 5/10
Current pair: 6/10
Current pair: 7/10
Current pair: 8/10
Current pair: 9/10


/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:184: RuntimeWarning:

invalid value encountered in divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:184: RuntimeWarning:

invalid value encountered in divide

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide



Updating parameters!
Current iter: 4, parameters: [ -5.          10.71428571  -0.71428571 -52.48026041   0.96575015] with loss: 0.16606758787507941
evaluate 10 pairs
Current pair: 0/10
Current pair: 1/10
Current pair: 2/10
Current pair: 3/10
Current pair: 4/10
Current pair: 5/10
Current pair: 6/10
Current pair: 7/10
Current pair: 8/10
Current pair: 9/10
Updating parameters!
Current iter: 5, parameters: [ -3.57142857  10.71428571  -0.71428571 -52.48026041   0.96575015] with loss: 0.16235134959048694
evaluate 10 pairs
Current pair: 0/10
Current pair: 1/10
Current pair: 2/10
Current pair: 3/10
Current pair: 4/10
Current pair: 5/10
Current pair: 6/10
Current pair: 7/10
Current pair: 8/10
Current pair: 9/10
Updating parameters!
Current iter: 6, parameters: [ -2.14285714  10.71428571  -2.14285714 -52.48026041   0.96575015] with loss: 0.16204878012286236
evaluate 10 pairs
Current pair: 0/10
Current pair: 1/10
Current pair: 2/10


In [ ]:
z_shift_correction.get_z_shift(smooth=False, allow_rotation=True)

In [ ]:
z_shift_correction.z_shift_correction(
    models=slices,
)

In [ ]:
method = 'PASTE'
align_Rotation = []
align_translation = []
for s in slices:
    R, t = solve_RT_by_correspondence(s.obsm['align_spatial_z_corrected'], s.obsm['perturbed_spatial'])
    align_Rotation.append(R)
    align_translation.append(t)
np.save(f'./mesh_corr_results/{method}_mesh_transformation.npy', {'align_Rotation':align_Rotation, 'align_translation':align_translation, 'best_parameters': z_shift_correction.best_parameters}, allow_pickle=True)